In [1]:
import requests
from pprint import pprint

session = requests.session()

In [2]:
from html import unescape
from bs4 import BeautifulSoup
import re
import random

In [75]:
response = session.get('https://www.kinopoisk.ru/reviews/type/comment/status/good/#list')

In [76]:
from fake_useragent import UserAgent
ua = UserAgent(verify_ssl=False)

Открываем страницу с 50 положительными отзывами

In [77]:
url = f'https://www.kinopoisk.ru/reviews/type/comment/status/good/period/month/perpage/50/#list'
req = session.get(url, headers={'User-Agent': ua.random})
page = req.text

In [78]:
soup = BeautifulSoup(page, 'html.parser')

Находим тег, внутри которого лежат тексты рецензий

In [79]:
reviews = soup.find_all('span', {'class': '_reachbanner_'})

In [80]:
reviews_str = str(reviews)

Чистим текст сначала от лишних знаков

In [81]:
cleaned_str = re.sub('\xa0', ' ', reviews_str)

Оставляем только кириллицу

In [82]:
reg = re.compile('[^а-яёА-ЯЁ ]')
pos_reviews = reg.sub(' ', cleaned_str)

Все то же самое делаем для негативных рецензий и тестовых

In [83]:
url = f'https://www.kinopoisk.ru/reviews/type/comment/status/bad/#list'
req = session.get(url, headers={'User-Agent': ua.random})
page = req.text

In [84]:
soup = BeautifulSoup(page, 'html.parser')

In [85]:
reviews_neg = soup.find_all('span', {'class': '_reachbanner_'})

In [86]:
reviews_str_neg = str(reviews_neg)

In [87]:
cleaned_str_neg = re.sub('\xa0', ' ', reviews_str_neg)

In [88]:
reg = re.compile('[^а-яёА-ЯЁ ]')
neg_reviews = reg.sub(' ', cleaned_str_neg)

In [89]:
url = f'https://www.kinopoisk.ru/reviews/type/comment/period/month/perpage/25/#list'
req = session.get(url, headers={'User-Agent': ua.random})
page = req.text

In [90]:
soup = BeautifulSoup(page, 'html.parser')

In [91]:
reviews_test = soup.find_all('span', {'class': '_reachbanner_'})

In [92]:
reviews_for_test = str(reviews_test)
r = reviews_for_test.split('<span class="_reachbanner_" itemprop="reviewBody">')

Здесь уже нужно, чтобы каждый текст был отдельно, поэтому разделяем их и кладем 10 рецензий в список

In [93]:
test = []
for rew in r:
    cleaned_str_test = re.sub('\xa0', ' ', rew)
    #print(cleaned_str_test)
    reg = re.compile('[^а-яёА-ЯЁ ]')
    test_reviews = reg.sub(' ', cleaned_str_test)
    #print(test_reviews)
    #while len(test) < 10:
    test.append(test_reviews)
test_list = []
test_list.append(test[1:11])
fin_list = test_list[0]
#print(fin_list[0])

Создаем вектор правильных ответов. 1 - положительный отзыв, 0 - отрицательный

In [123]:
label = [0, 0, 1, 0, 0, 1, 1, 1, 0, 1]

Датафрейм просто так, чтобы проверить, все ли в порядке 

In [96]:
import pandas as pd

In [97]:
df = pd.DataFrame(list(zip(fin_list, label)), columns =['review', 'label']) 

In [98]:
df

,review,label
0,Довольно вторичный фильм с заезженными клеше ...,1
1,Биографические фильмы это здорово Понятно ...,1
2,Это какая то фантастика Никто б...,0
3,Зарубежные ремейки успешных фильмов в США стоя...,0
4,но как говориться молчать не могу пан...,1
5,Замечательный одновременно легкий и глубокий ...,0
6,Первый раз я смотрел этот фильм в кинотеатре ...,1
7,Тема детей которые выросли без родителей ред...,0
8,Продюсеры Ветряной реки совместно с созда...,1
9,Каждая новая часть лучше предыдущей Как так...,0


In [99]:
import nltk
from nltk.tokenize import word_tokenize

In [100]:
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

Функция для приведения к нижнему регистру, токенизации, лемматизации

In [101]:
def text_prep(text):
    text = text.lower()
    tokenized = word_tokenize(text)
    lemmas = []
    for word in tokenized:
        ana = morph.parse(word)
        first = ana[0]
        lemma = first.normal_form
        lemmas.append(lemma)
    return(lemmas)

Делаем разные множества

In [102]:
positive_set = set(text_prep(pos_reviews))

In [103]:
negative_set = set(text_prep(neg_reviews))

In [104]:
p = positive_set.difference(negative_set) #уникальные элементы только в позитивных отзывах

In [105]:
n = negative_set.difference(positive_set) #уникальные элементы только в негативных отзывах

In [106]:
positive_list = text_prep(pos_reviews)

In [107]:
negative_list = text_prep(neg_reviews)

Ищем редкие слова и удаляем их из множеств

In [108]:
import collections
from collections import Counter

In [109]:
c = collections.Counter(positive_list)
c_list = c.items()

In [110]:
rare_words_pos = []
for el in c_list:
    if el[1] == 1 or el[1] == 2:
        rare_words_pos.append(el[0])

In [111]:
new_pos = []
for word in positive_list:
    if word not in rare_words_pos:
        new_pos.append(word)

In [112]:
c_neg = collections.Counter(negative_list)
c_list_neg = c_neg.items()

In [113]:
rare_words_neg = []
for el in c_list:
    if el[1] == 1 or el[1] == 2:
        rare_words_neg.append(el[0])

In [114]:
new_neg = []
for word in negative_list:
    if word not in rare_words_neg:
        new_neg.append(word)

In [115]:
new_pos_set = set(new_pos)

In [116]:
new_neg_set = set(new_neg)

Делаем сеты из списков, из которых с помощью Counter убраны слова, встречающиеся 1-2 раза

In [117]:
unique_positive = new_pos_set.difference(new_neg_set)

In [118]:
unique_negative = new_neg_set.difference(new_pos_set)

Функция, определяющая положительная рецензия или отрицательная

In [119]:
def super_definer(text):
    y = []
    positive_words = 0
    negative_words = 0
    for text in fin_list:
        text = text_prep(text)
        for word in text:
            if word in unique_positive:
                positive_words += 1
            if word in unique_negative:
                negative_words += 1
        if positive_words >= negative_words:
            y.append(1)
        if negative_words > positive_words:
            y.append(0)
    return(y)

In [120]:
for text in fin_list:
    y = super_definer(text)
print(y)

[0, 0, 0, 0, 0, 1, 1, 1, 1, 1]


In [121]:
from sklearn.metrics import accuracy_score

In [124]:
accuracy_score(y, label)

0.8

# Как можно улучшить программу

1. Например, исключить слова, которые в принципе являются частотными в языке. Это значит, они встречаются не только в положительных или отрицательных отзывах, но и в обычных не эмоционально окрашенных текстах.
2. Добавить в положительное или отрицательное множество словосочетания, помимо отдельных слов.
3. Возможно, применить метод TF-IDF, который поможет выделить значимые слова конкретно для положительных или отрицательных отзывов.